<a href="https://colab.research.google.com/github/Raph33AI/SonicSense-AI/blob/main/SonicSense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F # import torch.nn.functional
import torch.optim as optim
from torchvision import datasets, transforms

# 1. Définir le modèle (CNN simple)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Couches du réseau...
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        # Propagation avant...
        x = torch.relu(torch.max_pool2d(self.conv1(x), 2))
        x = torch.relu(torch.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = torch.relu(self.fc1(x))
        x = F.dropout(x, training=self.training) # Use F.dropout
        x = self.fc2(x)
        return torch.log_softmax(x, dim=1)

# 2. Charger les données (exemple MNIST)
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=False)

# 3. Instancier le modèle, la fonction de coût et l'optimiseur
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Entraîner le modèle
for epoch in range(10):  # Nombre d'époques
    for batch_idx, (data, target) in enumerate(train_loader):
        # Entraînement...
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

# 5. Évaluer le modèle
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        # Évaluation...
        output = model(data)
        _, predicted = torch.max(output, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = 100 * correct / total
print('Accuracy: {}'.format(accuracy))

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.331071
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.874986
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.382270
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.489720
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.640290
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.439420
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.229482
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.177342
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.350996
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.405223
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.251574
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.306478
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.203753
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.177620
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.228850
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.269507
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.198401
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.187443
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.195116
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.235656
Train Epoch:

In [24]:

import requests
import base64

# Vos identifiants d'application
CLIENT_ID = "dffbe0f471154691b105df32348a2378"
CLIENT_SECRET = "cd114e5ad1654451a84a1d3b9e32c7ee"

# Encodage des identifiants en base64
auth_string = f"{CLIENT_ID}:{CLIENT_SECRET}"
auth_bytes = auth_string.encode("utf-8")
auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

# URL du point de terminaison du jeton d'accès
token_url = "https://accounts.spotify.com/api/token"

# En-têtes de la requête
headers = {
    "Authorization": f"Basic {auth_base64}",
    "Content-Type": "application/x-www-form-urlencoded"
}

# Données de la requête
data = {"grant_type": "client_credentials"}

# Envoyer la requête POST
response = requests.post(token_url, headers=headers, data=data)

# Traiter la réponse
if response.status_code == 200:
    token_data = response.json()
    ACCESS_TOKEN = token_data["access_token"]
    print(f"Access token: {ACCESS_TOKEN}")
else:
    print(f"Erreur: {response.status_code}")
    print(response.text)

# Obtenir l'access token

if ACCESS_TOKEN:
    # En-têtes de la requête avec l'access token
    headers = {
        "Authorization": f"Bearer {ACCESS_TOKEN}"
    }

    #  ID du morceau que vous souhaitez récupérer
    track_id = "7KXjTSCq5nL1LoYtL7XAwS" # Remplacez par un ID Spotify valide
    track_url = f"https://api.spotify.com/v1/tracks/{track_id}"

    # Envoyer la requête GET pour récupérer les informations du morceau
    response = requests.get(track_url, headers=headers)

    # Traiter la réponse
    if response.status_code == 200:
        data = response.json()
        print(data)  # Afficher les métadonnées du morceau
    else:
        print(f"Erreur lors de la récupération des données du morceau: {response.status_code}")


Access token: BQDnbf-ReBvmloZlOVD2XkiDCYcCpH0EvWIhQRzwrnmG7rkGsctjSYwWNQFfcwzw2r13XlSDF7UzH_06ePJ-bYvibk8SsBtYdfiH2SvcH4zpzWm2oHM
{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2YZyLoL8N0Wb9xBt1NhZWg'}, 'href': 'https://api.spotify.com/v1/artists/2YZyLoL8N0Wb9xBt1NhZWg', 'id': '2YZyLoL8N0Wb9xBt1NhZWg', 'name': 'Kendrick Lamar', 'type': 'artist', 'uri': 'spotify:artist:2YZyLoL8N0Wb9xBt1NhZWg'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'M